In [ ]:
import os  # Для работы с директориями и переменными в окружении
import re  # Для работы с регулярными выражениями

# 1 - Парсинг текста из PDF

In [ ]:
# Путь к PDF документов
folder_path_to_pdf = "NPA"

## 1.1 - Использование PyMuPDF (fitz) для получения текста

In [ ]:
import fitz  # Для чтения текста из PDF

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with fitz.open(pdf_path) as doc:
            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                text += page.get_text()
    except Exception as e:
        print(f"Ошибка извлечения текста из {pdf_path}: {e}")

    return text

## 1.2 - Определение оглавления и основного текста документа

In [ ]:
def separate_text_from_toc(text):
    # Паттерн для разделения документа
    match = re.compile(r'(.*?)(?:приказом\s+ФСТЭК\s+России|Зарегистрировано\s+в\s+Министерстве\s+юстиции\s+Российской\s+Федерации|Одобрен\s+Советом\s+Федерации|к\s+приказу\s+ФСБ\s+России|Правительств(?:а|о))(.*)', re.DOTALL).search(text)
    if match:
        table_of_contents = match.group(1).strip()
        main_text = match.group(2).strip()
    else:
        table_of_contents = ""
        main_text = text.strip()

    return table_of_contents, main_text

## 1.3 - Очистка текста от лишней и повторяющейся информации

In [ ]:
def clean_text(text):
    # Удаление части текста, подготовленной АО "Кодекс"
    text = re.split(
        r'Электронный\s+текст\s+документа\s+подготовлен\s+АО\s+"Кодекс"\s+и\s+сверен\s+по:',
        text,
        flags=re.IGNORECASE
    )[0]

    # Удаление ненужных фраз и информации
    text = re.sub(
        r'(Документ предоставлен|www\.consultant\.ru|КонсультантПлюс|Страница\s+\d+\s+из\s+\d+\s|Дата сохранения: +\d{2}\.\d{2}\.\d{4})',
        '',
        text,
        flags=re.IGNORECASE
    ).strip()

    # Разделение текста на строки и удаление пустых строк
    lines_text = [line.strip() for line in text.split('\n') if line.strip()]

    # Удаление подчеркиваний и дефисов
    lines_text = [re.sub(r'(_+|-+)', '', line) for line in lines_text]

    # Объединение очищенных строк в одну строку
    cleaned_text = ' '.join(lines_text)
    
    return cleaned_text

## 1.4 - Извлечение идентификационных данных документа
- Оранжевое - организация;
- Фиолетовое - тип документа;
- Голубое - дата и номер документа.

<img src="figures/document_identification.jpg"/>

In [ ]:
def document_identification(text):
    # Паттерны для извлечения информации о документе
    document_pattern = re.compile(
        r"(ФЕДЕРАЛЬНАЯ\s+СЛУЖБА\s+(?:БЕЗОПАСНОСТИ\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ|ПО\s+ТЕХНИЧЕСКОМУ\s+И\s+ЭКСПОРТНОМУ\s+КОНТРОЛЮ)|ПРАВИТЕЛЬСТВО\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ|РОССИЙСКАЯ\s+ФЕДЕРАЦИЯ|ПРЕЗИДЕНТ\s+РОССИЙСКОЙ\s+ФЕДЕРАЦИИ)\s+"
        r"((?:ПРИКАЗ|ПОСТАНОВЛЕНИЕ|УКАЗ|ФЕДЕРАЛЬНЫЙ\s+ЗАКОН))\s+"
    )
    date_number_pattern = re.compile(
        r"(\d{1,2}\s+\w+\s+\d{4}\s+г(?:\.|ода)?)\s+(?:N|№)\s+(\d+)"
    )

    # Примеры:
    #   ФЕДЕРАЛЬНАЯ СЛУЖБА БЕЗОПАСНОСТИ РОССИЙСКОЙ ФЕДЕРАЦИИ ПРИКАЗ от 6 мая 2019 года N 196 Об утверждении Требований к средс...
    #   ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ ПОСТАНОВЛЕНИЕ от 8 февраля 2018 г. № 127 МОСКВА Об утверждении Правил категорирован...
    #   26 июля 2017 года N 187-ФЗ РОССИЙСКАЯ ФЕДЕРАЦИЯ ФЕДЕРАЛЬНЫЙ ЗАКОН О БЕЗОПАСНОСТИ КРИТИЧЕСКОЙ ИНФОРМАЦИОННОЙ ИНФРАСТРУК...
    #   ПРЕЗИДЕНТ РОССИЙСКОЙ ФЕДЕРАЦИИ УКАЗ от 30 марта 2022 г. N 166 О МЕРАХ ПО ОБЕСПЕЧЕНИЮ ТЕХНОЛОГИЧЕСКОЙ НЕЗАВИСИМОСТИ И Б...

    # Ищем соответствия в тексте
    document_match = document_pattern.search(text)
    date_number_match = date_number_pattern.search(text)

    if document_match and date_number_match:
        organization = document_match.group(1)
        document_type = document_match.group(2)
        date = date_number_match.group(1)
        document_number = date_number_match.group(2)

        return {
            'organization': organization,
            'document_type': document_type,
            'date': date,
            'document_number': document_number
        }
    else:
        print("Не удалось извлечь информацию из текста")
        
        return None

## 1.5 - Извлечение заголовков и (под)пунктов

In [ ]:
def extract_titles_and_texts(text):
    """
    Извлекает заголовки разделов с римскими цифрами и "Статья X" вместе с их текстом.
    """
    roman_pattern = re.compile(r"(?<=\b)([IVXLCDM]+\.\s+.+?)(?=\s+[IVXLCDM]+\.\s|\s*\d+\.\s|$)", re.MULTILINE | re.DOTALL)
    article_pattern = re.compile(r"(?<=\b)(Статья\s+\d+\.\s+.+?)(?=\s+Статья\s+\d+\.\s|\s*\d+\.\s|$)", re.MULTILINE | re.DOTALL)

    matches = list(re.finditer(roman_pattern, text)) + list(re.finditer(article_pattern, text))
    matches.sort(key=lambda x: x.start())

    results = []
    for i, match in enumerate(matches):
        title = match.group(1).strip()
        start_pos = match.end()

        end_pos = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        results.append((title, text[start_pos:end_pos].strip()))

    return results

def get_item_numbers(text):
    """
    Извлекает номера пунктов из текста.
    """
    pattern = re.compile(r"((?:\d+\.)+)\s*")
    match = pattern.match(text)
    if match:
        return match.group(1).strip().rstrip('.').split('.')
    
    return []

def extract_subpoints(text, counter_status=['0'], register_counter_status=0):
    """
    Извлекает подпункты из текста и обновляет их счетчики.
    """
    
    def update_counters(subpoint_number, counter_status, register_counter_status):
        """
        Обновляет счетчики для подуровней разделов.
        """
        if len(subpoint_number) > len(counter_status):
            register_counter_status += 1
            counter_status.append('0')
        elif len(subpoint_number) < len(counter_status):
            register_counter_status -= 1
            counter_status.pop()
    
        counter_status[register_counter_status] = str(int(counter_status[register_counter_status]) + 1)

        return counter_status, register_counter_status

    subpoint_pattern = re.compile(r"((?:\d+\.)+\s*.*?)(?=(?:\d+\.)+|$)", re.DOTALL)
    matches = re.findall(subpoint_pattern, text)
    subpoints = []

    for match in matches:
        subpoint = match.strip()
        subpoint_number = get_item_numbers(subpoint)
        if subpoint_number:
            counter_status, register_counter_status = update_counters(subpoint_number, counter_status, register_counter_status)
            if subpoint_number != counter_status:
                if subpoints:
                    subpoints[-1] += ' ' + subpoint
                    counter_status[register_counter_status] = str(int(counter_status[register_counter_status]) - 1)
            else:
                subpoints.append(subpoint)

    return subpoints, counter_status, register_counter_status

## 1.6 - Получение информации из PDF

In [ ]:
def process_pdf_file(pdf_file):
    """
    Обрабатывает PDF файл: извлекает текст, разделяет его на оглавление и основной текст,
    идентифицирует документ, извлекает заголовки и их подпункты.
    """
    text = clean_text(extract_text_from_pdf(pdf_file))
    doc_info, text = separate_text_from_toc(text)
    doc_info = document_identification(doc_info)
    titles_and_subpoints = {}

    # Поиск заголовков
    titles = extract_titles_and_texts(text)

    # Инициализация счетчиков подпунктов
    counter_subpoints = ['0']
    register_counter_status = 0

    if titles:
        for title, content in titles:
            print(f"\t({len(title)}) {title[:50]} [..]")
            if 'Статья ' in title:
                counter_subpoints = ['0']
                register_counter_status = 0
            subpoints, counter_subpoints, register_counter_status = extract_subpoints(content, counter_subpoints, register_counter_status)
            titles_and_subpoints[title] = subpoints
            for subpoint in subpoints:
                print(f"\t\t({len(subpoint)}) {subpoint[:50]} [..]")
    else:
        subpoints, counter_subpoints, register_counter_status = extract_subpoints(text, counter_subpoints, register_counter_status)
        for subpoint in subpoints:
            print(f"\t({len(subpoint)}) {subpoint[:50]} [..]")
        titles_and_subpoints["Без заголовка"] = subpoints

    return {
        'doc_info': doc_info,
        'titles_and_subpoints': titles_and_subpoints,
        'text': text
    }

def extract_info_from_pdfs(folder_path):
    """
    Извлекает информацию из всех PDF файлов в указанной папке.
    """
    pdf_texts = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            pdf_file = os.path.join(folder_path, filename)
            print(filename)
            pdf_data = process_pdf_file(pdf_file)
            pdf_texts[filename] = {
                'filename': filename,
                **pdf_data
            }
            print('-' * 100)
            
    return pdf_texts

pdf_texts = extract_info_from_pdfs(folder_path_to_pdf)

# 2 - Подготовка, создание графа

## 2.1 - Подготовка

### 2.1.1 - Разделение текста на предложения
Для разделения большого текста на предложения используется функция `sent_tokenize` и набор пунктуации из библиотеки `nltk`.

Если весь текст это предложение, то он будет разбит на чанки по знакам пунктуации.

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
# Набор пунктуации nltk
nltk.download('punkt')

In [ ]:
def split_text_by_punctuation(text, max_chunk_size):
    """
    Разделяет текст примерно в середине, с учетом размера чанка и пунктуации.
    """
    def find_split_index(text, max_chunk_size):
        # Найти индекс для разрыва текста около середины, но не превышая max_chunk_size
        mid = len(text) // 2
        # Найти ближайшую пунктуацию к середине текста
        split_points = re.finditer(r'[.,:;!?]\s*', text)
        best_split = None
        for point in split_points:
            if point.start() <= mid <= point.end():
                best_split = point.end()
                break
            if point.start() < mid:
                best_split = point.end()
            else:
                break
            
        return best_split if best_split and best_split <= max_chunk_size else mid

    chunks = []
    while len(text) > max_chunk_size:
        split_index = find_split_index(text, max_chunk_size)
        chunks.append(text[:split_index].strip())
        text = text[split_index:].strip()
    
    chunks.append(text)

    return chunks

### 2.1.2 - Суммаризация (Модель IlyaGusev/mbart_ru_sum_gazeta)
Если длина предложения превысит размер чанка, то из текста предложения достаётся смысловая выжимка.

Пример использования:
 - Пусть `article_text` это текст для обработки.

`input_ids = tokenizer([article_text], max_length=600, truncation=True, return_tensors="pt")["input_ids"].to("cuda")`

`output_ids = model.generate(input_ids=input_ids, no_repeat_ngram_size=4)[0]`

`summary = tokenizer.decode(output_ids, skip_special_tokens=True)`

In [ ]:
# Для инициализации модели для суммаризации текста
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [ ]:
summary_model_name = "IlyaGusev/mbart_ru_sum_gazeta"

summary_tokenizer = MBartTokenizer.from_pretrained(summary_model_name)  # Скачивание модели
summary_model = MBartForConditionalGeneration.from_pretrained(summary_model_name)  # Загрузка модели
summary_model.to("cuda")

In [ ]:
def summary_text(text, chunk_size, summary_tokenizer, summary_model):
    """
    Сжимает текст до размера чанка, используя модель суммаризации.
    """
    len_summary = chunk_size + 1
    previous_len = len(text)
    no_change_count = 0

    while len_summary > chunk_size and no_change_count < 3:
        input_ids = summary_tokenizer([text], max_length=chunk_size, truncation=True, return_tensors="pt")["input_ids"].to("cuda")
        output_ids = summary_model.generate(input_ids=input_ids, no_repeat_ngram_size=4)[0]
        new_text = summary_tokenizer.decode(output_ids, skip_special_tokens=True)
        
        if len(new_text) >= previous_len:
            no_change_count += 1
        else:
            no_change_count = 0
        
        text = new_text
        len_summary = len(text)
        previous_len = len(text)
    
    return text

### 2.1.3 - Доступ к LLM через API от HuggingFace (mistralai/Mixtral-8x7B-Instruct-v0.1)
Инициализация LLMs для дальнейшего использования этого в функции, которая конвертирует текст в узлы и отношения между ними.

Нужен [API ключ](https://huggingface.co/settings/tokens) в режиме `WRITE`.

Выбранные модели:
1) mistralai/Mistral-7B-Instruct-v0.1
2) mistralai/Mistral-7B-Instruct-v0.2
3) mistralai/Mistral-7B-Instruct-v0.3
4) mistralai/Mixtral-8x7B-Instruct-v0.1
5) meta-llama/Meta-Llama-3-8B-Instruct

In [ ]:
from getpass import getpass  # Для хранения секретных ключей

# Для инициализации и работы с LLM
from langchain_community.llms import HuggingFaceEndpoint
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass(prompt="Введите API ключ от HuggingFaceHub")

In [ ]:
models = [
    "mistralai/Mistral-7B-Instruct-v0.1",
    "mistralai/Mistral-7B-Instruct-v0.2",
    "mistralai/Mistral-7B-Instruct-v0.3",
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    "meta-llama/Meta-Llama-3-8B-Instruct"
]

def initialize_transformer(model_id):
    hg_llm = HuggingFaceEndpoint(repo_id=model_id)
    
    return LLMGraphTransformer(llm=hg_llm)

### 2.1.4 - Работа с Neo4j в AuraDB
Граф собирается при использовании Neo4j через облачную версию [AuraDB](https://neo4j.com/cloud/platform/aura-graph-database/).

In [ ]:
from neo4j import GraphDatabase  # Для подключения к среде Neo4j

In [ ]:
# Данные для входа в среду Neo4j
os.environ["NEO4J_URI"] = "neo4j+s://8a620302.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "CRQWPa75iQbJEfis2yH3DcVevljS17mloAB6Lm3Uoo0"

In [ ]:
class Neo4jGraph:
    def __init__(self):
        self.driver = GraphDatabase.driver(
            os.getenv("NEO4J_URI"),
            auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
        )

    def close(self):
        self.driver.close()

    def add_node(self, node_id, node_type, properties=None):
        if properties is None:
            properties = {}

        with self.driver.session() as session:
            if properties:
                session.run(f"""
                    MERGE (n:{node_type} {{id: $node_id}})
                    SET n += $properties
                    RETURN n
                """, node_id=node_id, properties=properties)
            else:
                session.run(f"""
                    MERGE (n:{node_type} {{id: $node_id}})
                    RETURN n
                """, node_id=node_id)

    def add_relationship(self, source_id, target_id, rel_type):
        with self.driver.session() as session:
            session.run(
                f"""
                MATCH (a {{id: $source_id}})
                MATCH (b {{id: $target_id}})
                MERGE (a)-[r:{rel_type}]->(b)
                """,
                source_id=source_id,
                target_id=target_id
            )

### 2.1.5 - Перевод текста на русский

In [ ]:
from deep_translator import GoogleTranslator  # Для перевода текста на русский

In [ ]:
def translate_to_rus(text):
    text = text.replace('_', ' ').replace('-', ' ')
    if text.isdigit():
        return text
    else:
        return GoogleTranslator(source='en', target='ru').translate(text)

### 2.1.6 - Подготовка ID и типа узла перед добавлением 

In [ ]:
def prepare_id_node(text):
    text = text.strip()
    if text != '':
        # Проверяем, соответствует ли строка шаблону заголовка в римском стиле
        if re.match(r'^[IVXLCDM]+\.\s+', text, re.IGNORECASE):
            return text.replace(' ', '_').replace('-', '_')
        else:
            return translate_to_rus(text).replace(' ', '_').replace('-', '_')
    else:
        return 'None'
    
def prepare_type_node(text):
    text = text.strip()
    if text != '':
        # Переводим текст и заменяем пробелы и дефисы на подчеркивания
        transformed_text = translate_to_rus(text).replace(' ', '_').replace('-', '_')
        # Преобразуем строку так, чтобы первый символ был в верхнем регистре, а остальные в нижнем
        transformed_text = transformed_text[0].upper() + transformed_text[1:].lower()
        return transformed_text
    else:
        return 'None'


# 2.2 - Создание графа

In [ ]:
from langchain_core.documents import Document  # Для конвертации текста в допустимый формат функции "convert_to_graph_documents"
import json  # Для логирования успешно завершенных этапов
import json_repair  # Для работы функции "convert_to_graph_documents"

In [ ]:
chunk_size = 500

In [ ]:
# Инициализируем граф
graph = Neo4jGraph()

# Загружаем существующий лог или создаём новый
log_file = 'processed_files_log.json'
if os.path.exists(log_file):
    with open(log_file, 'r', encoding='utf-8') as file:
        processed_log = json.load(file)
else:
    processed_log = {}

# Функция для обновления лога
def update_log(log_file, filename, paragraph, processed_log):
    if filename not in processed_log:
        processed_log[filename] = []
    processed_log[filename].append(paragraph)
    with open(log_file, 'w', encoding='utf-8') as file:
        json.dump(processed_log, file, ensure_ascii=False, indent=4)

# Функция для добавления базовых узлов
def add_basic_nodes(graph, doc):
    org_name = doc['organization']
    doc_type = doc['document_type']
    date = doc['date']
    doc_number = doc['document_number']
    
    org_id = org_name.replace(" ", "_").upper()
    graph.add_node(org_id, "Организация", {"название": org_name})
    
    graph.add_node(doc_number, "Документ", {
        "тип": doc_type,
        "дата": date
    })
    
    graph.add_relationship(org_id, doc_number, "ИЗДАННЫЙ")

# Функция для заполнения графа
def filling_in_graph(document, paragraph, graph, models):
    global hg_llm_transformer
    all_models_failed = True
    
    for model_id in models:
        try:
            hg_llm_transformer = initialize_transformer(model_id)
            graph_documents = hg_llm_transformer.convert_to_graph_documents(document)
            print(f"\n({len(graph_documents[0].nodes)}) Nodes: {graph_documents[0].nodes}")
            print(f"({len(graph_documents[0].relationships)}) Relationships: {graph_documents[0].relationships}")
            
            if len(graph_documents[0].nodes) == 0:
                print(f"\n{model_id} выдала пустой граф.")
                continue
            
            all_models_failed = False
            
            for graph_document in graph_documents:
                for node in graph_document.nodes:
                    graph.add_node(prepare_id_node(node.id), prepare_type_node(node.type))
                    graph.add_relationship(prepare_id_node(paragraph), prepare_id_node(node.id), 'СОДЕРЖАНИЕ')
                for relationship in graph_document.relationships:
                    relationship_source_id = prepare_id_node(relationship.source.id)
                    relationship_target_id = prepare_id_node(relationship.target.id)
                    if relationship_source_id != relationship_target_id:
                        graph.add_relationship(
                            relationship_source_id,
                            relationship_target_id,
                            translate_to_rus(relationship.type).upper().replace(' ', '_')
                        )
            
            print('-' * 100)
            break  # Выход из цикла при успешной обработке
        except Exception as e:
            print(f"\nОшибка при использовании модели {model_id}: {e}")
            continue
    
    if all_models_failed:
        print("\nВсе модели выдали плохой результат. Пропуск текста.")
        print('-' * 100)
        return False
    return True


# Основной цикл обработки файлов
for filename in pdf_texts:
    document_info = pdf_texts[filename]['doc_info']
    if document_info is not None:
        add_basic_nodes(graph, document_info)
        doc_number = document_info['document_number']

        for titles_and_subpoints in pdf_texts[filename]['titles_and_subpoints']:
            title = f'{titles_and_subpoints.replace(" ", "_").replace(".", "")}_{doc_number}'

            if 'Без_заголовка' not in title:
                graph.add_node(prepare_id_node(title), 'Заголовок_пункта')
                graph.add_relationship(prepare_id_node(doc_number), prepare_id_node(title), 'СОДЕРЖАНИЕ')

            for text in pdf_texts[filename]['titles_and_subpoints'][titles_and_subpoints]:
                paragraph = f"{'_'.join(get_item_numbers(text))}_({prepare_id_node(doc_number)})"

                if filename in processed_log and paragraph in processed_log[filename]:
                    print(f"Пропуск обработанного пункта: {paragraph}")
                    continue

                graph.add_node(prepare_id_node(paragraph), 'Пункт')
                if 'Без_заголовка' not in title:
                    # Привязываем заголовок к (под)пункту
                    graph.add_relationship(prepare_id_node(title), prepare_id_node(paragraph), 'СОДЕРЖАНИЕ')
                else:
                    # Привязываем (под)пункт к номеру документа организации
                    graph.add_relationship(prepare_id_node(doc_number), prepare_id_node(paragraph), 'СОДЕРЖАНИЕ')

                text = summary_text(text, chunk_size, summary_tokenizer, summary_model)

                if len(text) > chunk_size:
                    text = sent_tokenize(text)
                    if len(text) > 2:
                        text = split_text_by_punctuation(text[0], chunk_size)

                if isinstance(text, str):
                    if filling_in_graph([Document(page_content=text)], paragraph, graph, models):
                        update_log(log_file, filename, paragraph, processed_log)
                elif isinstance(text, list):
                    for text_small in text:
                        if len(text_small) > chunk_size:
                            text_small = summary_text(text_small, chunk_size, summary_tokenizer, summary_model)
                            
                        if filling_in_graph([Document(page_content=text_small)], paragraph, graph, models):
                            update_log(log_file, filename, paragraph, processed_log)

# Закрываем соединение с графом
graph.close()